In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.ensemble as ske
import numpy as np
import sklearn.metrics as skm
import sklearn.model_selection as skms
import sklearn.tree as skt
import sklearn.linear_model as sklm
import sklearn.discriminant_analysis as skda
from sklearn.model_selection import train_test_split

sys.path.append("..")
from utils.loading_data  import load_to_df_from_csv
df = load_to_df_from_csv("../data/train.csv")

for i in range(len(df)): #Loop to increment the lead words into female words or male words

    if df.loc[i,"Lead"]=="Female":

        df.loc[i,"Number words female"]=df.loc[i,"Number words female"]+df.loc[i,"Number of words lead"]
    else:
        df.loc[i,"Number words male"]=df.loc[i,"Number words male"]+df.loc[i,"Number of words lead"]
df["difference_words_m_f"]=df.iloc[:,7]-df.iloc[:,0] #Create new feature, the difference in words spoken between males and females.

In [2]:
#This cell implements Adaboost with base estimators logistic regression, decision tree and random forest
X=df.iloc[:,0:13]
y=df.iloc[:,13]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2) #Random split data 75,25

X_train.reset_index(drop=True,inplace=True) #Reset index 

y_train.reset_index(drop=True,inplace=True) #Reset index


logistic_object=sklm.LogisticRegression(max_iter=10000) #A logistic model, with default solver

decisiontree_object=skt.DecisionTreeClassifier(max_depth=10,criterion="entropy") #A decision tree modele

randomforest_object=ske.RandomForestClassifier(n_estimators=100,criterion="entropy") #Random forest

objects=[logistic_object,decisiontree_object,randomforest_object] 

X=X_train

y=y_train

k_folds_object=skms.StratifiedKFold(n_splits=10,shuffle=True,random_state=2) #A stratified K-fold object is created, to evaluate each model on 70% of the 
#original data

for x in objects: #Iterate through each object
    boosting_object=ske.AdaBoostClassifier(base_estimator=x,n_estimators=100)
    acc_score = []
    k_folds_object=skms.StratifiedKFold(n_splits=10,shuffle=True,random_state=2)

    for train_index , test_index in k_folds_object.split(X,y): #Stratified K-fold
        X_train1 , X_test1 = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train1 , y_test1 = y[train_index] , y[test_index]
        
        boosting_object.fit(X_train1,y_train1)

        pred_values = boosting_object.predict(X_test1)
        
        acc = skm.accuracy_score(pred_values , y_test1)
        acc_score.append(acc)
    avg_acc_score = np.mean(acc_score)
    print("Average accuracy for:",x,avg_acc_score) #Largest stratified k-fold accuracy comes from a boosted LR model.


Average accuracy for: LogisticRegression(max_iter=10000) 0.9191475191475191
Average accuracy for: DecisionTreeClassifier(criterion='entropy', max_depth=10) 0.8562104562104563
Average accuracy for: RandomForestClassifier(criterion='entropy') 0.8998834498834498


In [3]:
#Cell to evaluate the boosted LR model from above on unseen test data
boosted_logistic_object=ske.AdaBoostClassifier(base_estimator=logistic_object,n_estimators=10)
boosted_logistic_object.fit(X_train,y_train)

#Train accuracy
pred=boosted_logistic_object.predict(X_train)
print(skm.accuracy_score(pred,y_train))

#Test accuracy
pred=boosted_logistic_object.predict(X_test)
print(skm.accuracy_score(pred,y_test))

0.9255455712451861
0.8807692307692307
